In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

김치찌개 레시피 크롤링

In [4]:
import requests
import json
from bs4 import BeautifulSoup

def food_info(name):
    '''
    This function gives you food information for the given input.

    PARAMETERS
        - name(str): name of Korean food in Korean ex) food_info("김치찌개")
    RETURN
        - res(list): list of dict that containing info for some Korean food related to 'name'
            - res['name'](str): name of food
            - res['ingredients'](str): ingredients to make the food
            - res['recipe'](list[str]): contain recipe in order
    '''
    url = f"https://www.10000recipe.com/recipe/list.html?q={name}"
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
    else : 
        print("HTTP response error :", response.status_code)
        return
    
    food_list = soup.find_all(attrs={'class':'common_sp_link'})
    food_id = food_list[0]['href'].split('/')[-1]
    new_url = f'https://www.10000recipe.com/recipe/{food_id}'
    new_response = requests.get(new_url)
    if new_response.status_code == 200:
        html = new_response.text
        soup = BeautifulSoup(html, 'html.parser')
    else : 
        print("HTTP response error :", response.status_code)
        return
    
    food_info = soup.find(attrs={'type':'application/ld+json'})
    result = json.loads(food_info.text)
    title = result['name']
    people = result['recipeYield']
    ingredient = ','.join(result['recipeIngredient'])
    recipe = [result['recipeInstructions'][i]['text'] for i in range(len(result['recipeInstructions']))]
    for i in range(len(recipe)):
        recipe[i] = f'{i+1}. ' + recipe[i]
    
    res = {
        'name': name,
        'title':title,
        'people': people,
        'ingredients': ingredient,
        'recipe': recipe
    }

    return res

# Example usage:
name = ["편스토랑","백종원"]
food_info_result = food_info(name)
print(food_info_result)

{'name': ['편스토랑', '백종원'], 'title': '백종원 오이무침 새콤달콤 맛있게~', 'people': '4 servings', 'ingredients': '오이 2개,양파 1/2개,고추장 1T,고춧가루 1T,설탕 1T,올리고당 1T,다진마늘 1/2T,간장 1T,식초 1.5T,참기름 1T,통깨 1T', 'recipe': ['1. 오이는 동글동글 모양살려 썰어 소금에 잠시 절여 둡니다', '2. 절인다기 보다는 양념 준비하는 동안 잠시 소금에 절인다 생각하면 됩니다', '3. 고추장(1T), 고춧가루(1T), 설탕(1T), 올리고당(1T) 다진마늘(1/2T), 간장(1T), 식초(1.5T), 참기름(1T), 통깨(1T) 섞어 양념장 만들어요', '4. 물기 꼭 짜 주고', '5. 슬라이스한 양파도 넣어요', '6. 준비한 양념장 넣고 무쳐 냅니다', '7. 새콤달콤 맛있는 오이무침 완성입니다 수분을 꼭 짜고 무친 것이라 꼬들꼬들 아삭함이 좋은 오이무침 입니다 맛있어요 ㅎ']}


레시피 Top100 크롤링

In [ ]:
from bs4 import BeautifulSoup
import requests
url = "https://www.10000recipe.com/ranking/home_new.html"
html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")
menus = soup.find_all("li", {"class": "common_sp_list_li"}) # 레시피 목록을 포함하는 모든 요소 찾기
for menu in menus: # 각각의 레시피 요소에 대해 반복
    title = menu.find("div", {"class": "common_sp_caption_tit"}).text # 요리의 제목 추출
    filename = title.replace(" ", "")[:8] # 요리 제목에서 공백을 제거하고 파일 이름으로 사용할 부분 선택
    link = menu.find("a", {"class": "common_sp_link"}) # 요리의 상세 페이지 링크 가져오기
    new_url = "http://10000recipe.com" + link['href']
    sub_html = requests.get(new_url).text # 각 레시피의 상세 페이지에 접근하여 HTML 가져오기
    sub_soup = BeautifulSoup(sub_html, "html.parser")
    ingre3 = sub_soup.find("div", {"class": "ready_ingre3"}, {"id": "divConfirmedMaterialArea"}) # 상세 페이지에서 재료 찾기
    ingre = ingre3.find_all("li")
    lists = "" # 재료를 리스트에 추가하고 출력
    for i in ingre:
        lists += i.text.strip() + ", "
        print(i.text)
    steps = sub_soup.find_all("div", {"class": "view_step_cont"}) # 상세 페이지에서 조리방법 찾기
    for s in steps: # 조리방법 출력
        print(s.find("div", {"class": "media-body"}).text)
import csv
menus = soup.find_all("li", {"class": "common_sp_list_li"})
with open('recipe.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Ingredients', 'Steps'])
    for menu in menus:
        title = menu.find("div", {"class": "common_sp_caption_tit"}).text
        filename = title.replace(" ", "")[:8]
        link = menu.find("a", {"class": "common_sp_link"})
        new_url = "http://10000recipe.com" + link['href']
        sub_html = requests.get(new_url).text
        sub_soup = BeautifulSoup(sub_html, "html.parser")
        ingre3 = sub_soup.find("div", {"class": "ready_ingre3"}, {"id": "divConfirmedMaterialArea"})
        ingre = ingre3.find_all("li")
        ingre_list = ", ".join(i.text.strip() for i in ingre)
        steps = sub_soup.find_all("div", {"class": "view_step_cont"})
        steps_list = "\n".join(s.find("div", {"class": "media-body"}).text.strip() for s in steps)
        writer.writerow([title, ingre_list, steps_list])        

In [28]:
from bs4 import BeautifulSoup
import requests

url = "https://www.10000recipe.com/ranking/home_new.html"

html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")

In [47]:
menus = soup.find_all("li", {"class": "common_sp_list_li"})
for menu in menus:
    title = menu.find("div", {"class": "common_sp_caption_tit"}).text
    filename = title.replace(" ", "")[:8]

    link = menu.find("a", {"class": "common_sp_link"})
    new_url = "http://10000recipe.com" + link['href']

    sub_html = requests.get(new_url).text
    sub_soup = BeautifulSoup(sub_html, "html.parser")

    ingre3 = sub_soup.find("div", {"class": "ready_ingre3"}, {"id": "divConfirmedMaterialArea"})
    ingre = ingre3.find_all("li")
    lists = ""
    for i in ingre:
        lists += i.text.strip() + ", "
        print(i.text)

    steps = sub_soup.find_all("div", {"class": "view_step_cont"})
    for s in steps:
        print(s.find("div", {"class": "media-body"}).text)




                                                                오이                                                                                                            

10개 
구매




                                                                굵은소금                                                                                손질/절임용


12T 
구매




                                                                부추                                                                                                            

1/2단 
구매




                                                                양파                                                                                                            

1개 
구매




                                                                물                                                                                                            

12컵 
구매




                                                                굵은고춧가루       

In [48]:
import csv

menus = soup.find_all("li", {"class": "common_sp_list_li"})
with open('recipe.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Ingredients', 'Steps'])

    for menu in menus:
        title = menu.find("div", {"class": "common_sp_caption_tit"}).text
        filename = title.replace(" ", "")[:8]

        link = menu.find("a", {"class": "common_sp_link"})
        new_url = "http://10000recipe.com" + link['href']

        sub_html = requests.get(new_url).text
        sub_soup = BeautifulSoup(sub_html, "html.parser")

        ingre3 = sub_soup.find("div", {"class": "ready_ingre3"}, {"id": "divConfirmedMaterialArea"})
        ingre = ingre3.find_all("li")
        ingre_list = ", ".join(i.text.strip() for i in ingre)

        steps = sub_soup.find_all("div", {"class": "view_step_cont"})
        steps_list = "\n".join(s.find("div", {"class": "media-body"}).text.strip() for s in steps)

        writer.writerow([title, ingre_list, steps_list])

레시피 Top100 전처리

In [24]:
df = pd.read_csv("./recipe.csv", encoding='utf-8')
df

,Title,Ingredients,Steps
0,오이 소박이 무침 황금 레시피/오이 김치 담그는 법/오이 부추 무침,오이 ...,오이는 소금(6큰술)로 겉을 충분히 문질러 씻은 후 행궈주세요\n양끝을 잘라낸 후 ...
1,"즉석 오이무침 만드는 법, 아삭아삭 상큼한 오이무침, 밑반찬",오이 ...,오이는 굵은 소금으로 문질러 가시부분을 제거하거나 칼등으로 제거 해 주고 동글동글 ...
2,백종원레시피로 만든 콩나물무침으로 밥 한 끼 뚝딱 ~,콩나물 ...,콩나물은 흐르는 물에 여러 번 조심스레 씻어준 뒤 체에 밭쳐 물기를 빼둡니다.\n당...
3,두부조림 양념장 만드는 법,두부 ...,먹기 좋게 한 입 크기로 도톰하게 썰어주고 키친타월을 이용해 물기를 제거하고 그 위...
4,"오징어 볶음, 향과 맛이 일품! 백종원 오징어 볶음",오징어 ...,"양배추, 당근, 양파, 파(준비량의 반)는 길쭉하고 굵게, 고추도 어슷큼직하게 썹니..."
...,...,...,...
95,"백종원오이무침,초간단오이무침 만드는 법",오이 ...,"백종원오이무침,초간단오이무침 만드는 법\n재료:오이2개,양파1/2개,굵은소금1스푼 ..."
96,생생정보통 황금레시피-김밥만들기,밥 ...,쌀과 물의 비율은 1:1 저는 쌀의 양을 머그컵으로 한 컵 반 정도 했는데 6줄 싸...
97,꼬막무침 삶는방법부터 양념장까지 완벽하게!!,꼬막 ...,꼬막은 깨지거나 흠이 없고\n색과 골이 선명한 것이 좋아요!\n1. 장갑을 끼고 소...
98,김치 볶음밥 완전 새로운 방식!,김치 ...,김치 1컵을 잘게 썬다. 새김치는 안되지만 살짝 익은 정도의 김치면 가능. 레시피에...


In [25]:
df['인분'] = '2인분'
df

,Title,Ingredients,Steps,인분
0,오이 소박이 무침 황금 레시피/오이 김치 담그는 법/오이 부추 무침,오이 ...,오이는 소금(6큰술)로 겉을 충분히 문질러 씻은 후 행궈주세요\n양끝을 잘라낸 후 ...,2인분
1,"즉석 오이무침 만드는 법, 아삭아삭 상큼한 오이무침, 밑반찬",오이 ...,오이는 굵은 소금으로 문질러 가시부분을 제거하거나 칼등으로 제거 해 주고 동글동글 ...,2인분
2,백종원레시피로 만든 콩나물무침으로 밥 한 끼 뚝딱 ~,콩나물 ...,콩나물은 흐르는 물에 여러 번 조심스레 씻어준 뒤 체에 밭쳐 물기를 빼둡니다.\n당...,2인분
3,두부조림 양념장 만드는 법,두부 ...,먹기 좋게 한 입 크기로 도톰하게 썰어주고 키친타월을 이용해 물기를 제거하고 그 위...,2인분
4,"오징어 볶음, 향과 맛이 일품! 백종원 오징어 볶음",오징어 ...,"양배추, 당근, 양파, 파(준비량의 반)는 길쭉하고 굵게, 고추도 어슷큼직하게 썹니...",2인분
...,...,...,...,...
95,"백종원오이무침,초간단오이무침 만드는 법",오이 ...,"백종원오이무침,초간단오이무침 만드는 법\n재료:오이2개,양파1/2개,굵은소금1스푼 ...",2인분
96,생생정보통 황금레시피-김밥만들기,밥 ...,쌀과 물의 비율은 1:1 저는 쌀의 양을 머그컵으로 한 컵 반 정도 했는데 6줄 싸...,2인분
97,꼬막무침 삶는방법부터 양념장까지 완벽하게!!,꼬막 ...,꼬막은 깨지거나 흠이 없고\n색과 골이 선명한 것이 좋아요!\n1. 장갑을 끼고 소...,2인분
98,김치 볶음밥 완전 새로운 방식!,김치 ...,김치 1컵을 잘게 썬다. 새김치는 안되지만 살짝 익은 정도의 김치면 가능. 레시피에...,2인분


In [26]:
# 컬럼명 변경
df = df.rename(columns={'Title':'제목', 'Ingredients':'재료', 'Steps':'조리방법'})

In [27]:
# 컬럼 순서 변경
new_order = ['제목', '인분', '재료', '조리방법']
df = df[new_order]

In [28]:
df.head()

,제목,인분,재료,조리방법
0,오이 소박이 무침 황금 레시피/오이 김치 담그는 법/오이 부추 무침,2인분,오이 ...,오이는 소금(6큰술)로 겉을 충분히 문질러 씻은 후 행궈주세요\n양끝을 잘라낸 후 ...
1,"즉석 오이무침 만드는 법, 아삭아삭 상큼한 오이무침, 밑반찬",2인분,오이 ...,오이는 굵은 소금으로 문질러 가시부분을 제거하거나 칼등으로 제거 해 주고 동글동글 ...
2,백종원레시피로 만든 콩나물무침으로 밥 한 끼 뚝딱 ~,2인분,콩나물 ...,콩나물은 흐르는 물에 여러 번 조심스레 씻어준 뒤 체에 밭쳐 물기를 빼둡니다.\n당...
3,두부조림 양념장 만드는 법,2인분,두부 ...,먹기 좋게 한 입 크기로 도톰하게 썰어주고 키친타월을 이용해 물기를 제거하고 그 위...
4,"오징어 볶음, 향과 맛이 일품! 백종원 오징어 볶음",2인분,오징어 ...,"양배추, 당근, 양파, 파(준비량의 반)는 길쭉하고 굵게, 고추도 어슷큼직하게 썹니..."


In [29]:
# '조리방법' 컬럼 안에 있는 값들에 '\n'이 나오면 줄바꿈 후 숫자.을 추가하기
df['조리방법'] = df['조리방법'].str.split('\n')
for i, steps in enumerate(df['조리방법']):
    df['조리방법'][i] = '\n'.join([f"{i+1}. {step}" for i, step in enumerate(steps) if step.strip() != ''])

df

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_16664\190544529.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['조리방법'][i] = '\n'.join([f"{i+1}. {step}" for i, step in enumerate(steps) if step.strip() != ''])
C:\Users\tjoeun\AppData\Loca

,제목,인분,재료,조리방법
0,오이 소박이 무침 황금 레시피/오이 김치 담그는 법/오이 부추 무침,2인분,오이 ...,1. 오이는 소금(6큰술)로 겉을 충분히 문질러 씻은 후 행궈주세요\n2. 양끝을 ...
1,"즉석 오이무침 만드는 법, 아삭아삭 상큼한 오이무침, 밑반찬",2인분,오이 ...,1. 오이는 굵은 소금으로 문질러 가시부분을 제거하거나 칼등으로 제거 해 주고 동글...
2,백종원레시피로 만든 콩나물무침으로 밥 한 끼 뚝딱 ~,2인분,콩나물 ...,1. 콩나물은 흐르는 물에 여러 번 조심스레 씻어준 뒤 체에 밭쳐 물기를 빼둡니다....
3,두부조림 양념장 만드는 법,2인분,두부 ...,1. 먹기 좋게 한 입 크기로 도톰하게 썰어주고 키친타월을 이용해 물기를 제거하고 ...
4,"오징어 볶음, 향과 맛이 일품! 백종원 오징어 볶음",2인분,오징어 ...,"1. 양배추, 당근, 양파, 파(준비량의 반)는 길쭉하고 굵게, 고추도 어슷큼직하게..."
...,...,...,...,...
95,"백종원오이무침,초간단오이무침 만드는 법",2인분,오이 ...,"1. 백종원오이무침,초간단오이무침 만드는 법\n2. 재료:오이2개,양파1/2개,굵은..."
96,생생정보통 황금레시피-김밥만들기,2인분,밥 ...,1. 쌀과 물의 비율은 1:1 저는 쌀의 양을 머그컵으로 한 컵 반 정도 했는데 6...
97,꼬막무침 삶는방법부터 양념장까지 완벽하게!!,2인분,꼬막 ...,1. 꼬막은 깨지거나 흠이 없고\n2. 색과 골이 선명한 것이 좋아요!\n3. 1....
98,김치 볶음밥 완전 새로운 방식!,2인분,김치 ...,1. 김치 1컵을 잘게 썬다. 새김치는 안되지만 살짝 익은 정도의 김치면 가능. 레...
